<a href="https://colab.research.google.com/github/kuheli31/Fake-News-Detector/blob/main/Fake_News_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import re
import string
import requests
from requests.utils import quote
from urllib.parse import quote

# Enter the news to verify if its real or fake

In [52]:
import spacy

# Load SpaCy model for keyword extraction
nlp = spacy.load("en_core_web_sm")

# Step 1: Get news input
news = input("Enter the news to verify if it is real or fake: ")

# Step 2: Process the text
doc = nlp(news)

# Step 3: Extract potential keywords — proper nouns or nouns that matter
keywords = [token.text for token in doc if token.pos_ in ["PROPN", "NOUN"] and len(token.text) > 2]

# Step 4: Remove duplicates while preserving order
seen = set()
keywords = [k for k in keywords if not (k.lower() in seen or seen.add(k.lower()))]

# Step 5: Keep only top 4 most important words (based on position and relevance)
news_check = keywords[:4]

print("Extracted Keywords for Search:", " ".join(news_check))


Enter the news to verify if it is real or fake: India poised to play key role at G20 Summit in Johannesburg - New Delhi, Nov 2 India, as the world's fastest-growing major economy, is poised to play a key role ... Get Latest News on Business only on lokmattimes.com
Extracted Keywords for Search: India role G20 Summit


# Data Collection

**G-News API (*Real News*)**

In [28]:
#Real time news api (https://gnews.io/dashboard) -- 100 requests per day
news_api_key = 'd2b27a1e3f818564545508b6c744d24a'
#Endpoints are -> {search , top-headlines}
endpoint = 'search'
#Parameters are -> {The topic the user want to search}
question = " ".join(news_check)
parameters = f'q={quote(question)}&lang=en&max=5'
#Example (try this link to understand the url)-> https://gnews.io/api/v4/top-headlines?q=modi&lang=en&max=5&apikey=d2b27a1e3f818564545508b6c744d24a
news_api_url = f'https://gnews.io/api/v4/{endpoint}?{parameters}&apikey={news_api_key}'

response = requests.get(news_api_url)
data = response.json()

if 'articles' in data:
    for article in data['articles']:
        print(f"Published Time: {article['publishedAt']}")
        print(f"Title: {article['title']}")
        print(f"Description: {article['description']}")
        print(f"Image: {article['image']}")
        print(f"Content: {article['content']}")
        print(f"Source Name: {article['source']['name']}")
        print(f"Source URL: {article['source']['url']}")
        print(f"Source Country: {article['source']['country']}")
        print(f"URL: {article['url']}\n")
 # Extract article details into a list of dicts
    articles_data = []
    for article in data['articles']:
        articles_data.append({
            "Published Time": article.get('publishedAt'),
            "Title": article.get('title'),
            "Description": article.get('description'),
            "Image URL": article.get('image'),
            "Content": article.get('content'),
            "Source Name": article['source'].get('name'),
            "Source URL": article['source'].get('url'),
            "Source Country": article['source'].get('country'),
            "Article URL": article.get('url')
        })

    # Create DataFrame
    df = pd.DataFrame(articles_data)

    # Save as CSV
    df.to_csv('gnews.csv', index=False, encoding='utf-8')

    print("✅ Articles successfully saved to 'gnews.csv'")

else:
    print(data)

Published Time: 2025-11-02T11:37:15Z
Title: India poised to play key role at G20 Summit in Johannesburg
Description: India poised to play key role at G20 Summit in Johannesburg - New Delhi, Nov 2 India, as the world's fastest-growing major economy, is poised to play a key role ... Get Latest News on Business only on lokmattimes.com
Image: https://d3pc1xvrcw35tl.cloudfront.net/ln/feedImages/1200x900/202511023561587_202511.jpg
Content: New Delhi, Nov 2 India, as the world's fastest-growing major economy, is poised to play a key role in steering the forthcoming G20 Summit in Johannesburg in November 2025 toward realistic and inclusive solutions amid the global slowdown and trade ten... [3586 chars]
Source Name: Lokmat Times
Source URL: https://www.lokmattimes.com
Source Country: in
URL: https://www.lokmattimes.com/business/india-poised-to-play-key-role-at-g20-summit-in-johannesburg-1/

Published Time: 2025-11-02T11:37:15Z
Title: India poised to play key role at G20 Summit in Johannesburg


**Twitter API**

In [30]:
# Importing necessary libraries for X (Twitter) API interaction
import tweepy
import pandas as pd

# Hardcoded Bearer Token
X_API_BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAKCJ4QEAAAAAcEQy2tIw03BHpMFUg3fg9sbpGu8%3Db2Q2D7tbJUI2A0fJjOOMz69iKILmCmXeBYwVNNoxOaty3y2Oj2"

# Searching in tweets
question = " ".join(news_check).strip()

# --- Main Execution Block ---
if not X_API_BEARER_TOKEN or not question:
    print("ERROR: Both the Bearer Token and the search query are required.")
else:
    try:
        # Initialize Tweepy Client
        client = tweepy.Client(X_API_BEARER_TOKEN)

        # API Call with image and profile image support
        response = client.search_recent_tweets(
            query=f'{question} -is:retweet has:images lang:en',
            max_results=10,
            tweet_fields=['created_at', 'public_metrics', 'attachments'],
            user_fields=['name', 'username', 'verified', 'profile_image_url'],
            media_fields=['url', 'preview_image_url', 'type'],
            expansions=['author_id', 'attachments.media_keys']
        )

        if not response or not response.data:
            print("\nNo recent tweets found for that query.")
        else:
            tweets = response.data
            users = {user['id']: user for user in response.includes.get('users', [])} if response.includes else {}
            media = {m['media_key']: m for m in response.includes.get('media', [])} if response.includes else {}

            print("\n--- Found Tweets ---\n")

            # --- Create list for DataFrame ---
            tweet_rows = []

            for tweet in tweets:
                author = users.get(tweet.author_id)
                media_urls = []

                # Extract media URLs
                if hasattr(tweet, 'attachments') and 'media_keys' in tweet.attachments:
                    for key in tweet.attachments['media_keys']:
                        if key in media and media[key].type == 'photo':
                            media_urls.append(media[key].url)

                tweet_url = f"https://x.com/{author.username}/status/{tweet.id}" if author else None
                author_url = f"https://x.com/{author.username}" if author else None

                # --- Print Output ---
                print(f"Published Time: {tweet.created_at}")
                print(f"Tweet Text: {tweet.text}")
                if author:
                    print(f"Author Name: {author.name} (@{author.username})")
                    print(f"Author Verified: {author.verified}")
                    print(f"Author Profile Image: {author.profile_image_url}")
                    print(f"Author URL: {author_url}")
                print(f"Tweet URL: {tweet_url}")

                if media_urls:
                    print("Image URLs:")
                    for img in media_urls:
                        print(f"- {img}")
                else:
                    print("No images found.")

                # Engagement metrics
                if tweet.public_metrics:
                    print(f"Likes: {tweet.public_metrics.get('like_count', 0)}")
                    print(f"Retweets: {tweet.public_metrics.get('retweet_count', 0)}")
                    print(f"Replies: {tweet.public_metrics.get('reply_count', 0)}")
                print("\n")

                # --- Add tweet info for DataFrame ---
                tweet_rows.append({
                    "Published Time": tweet.created_at,
                    "Tweet Text": tweet.text,
                    "Author Name": author.name if author else None,
                    "Author Username": author.username if author else None,
                    "Author Verified": author.verified if author else None,
                    "Author Profile Image": author.profile_image_url if author else None,
                    "Author URL": author_url,
                    "Tweet URL": tweet_url,
                    "Image URLs": ", ".join(media_urls) if media_urls else None,
                    "Likes": tweet.public_metrics.get("like_count") if tweet.public_metrics else None,
                    "Retweets": tweet.public_metrics.get("retweet_count") if tweet.public_metrics else None,
                    "Replies": tweet.public_metrics.get("reply_count") if tweet.public_metrics else None
                })

            # --- Convert tweets to DataFrame and save to CSV ---
            df_tweets = pd.DataFrame(tweet_rows)
            df_tweets.to_csv('tweets.csv', index=False, encoding='utf-8')

            print("✅ Tweets successfully saved to 'tweets.csv'")

    except tweepy.errors.Unauthorized:
        print("\n--- AUTHENTICATION ERROR ---")
        print("The Bearer Token you provided is invalid. Please check your credentials on the X Developer Portal.")
    except tweepy.errors.BadRequest:
        print("\n--- INVALID SEARCH QUERY ---")
        print("Your search query is invalid or too complex. Please check the format.")
    except Exception as e:
        print(f"\nAn unexpected error occurred.")
        print(f"Error details: {e}")



--- Found Tweets ---

Published Time: 2025-11-02 10:47:51+00:00
Tweet Text: Seeing most Nigerians talking about this image, the meeting took place on the sidelines of the G20 Summit in New Delhi, India, in September 2023. They both attended the G20 Summit, Mr President did not go to a meeting with any US president either Biden or Trump https://t.co/Xsucj3LSje https://t.co/aEkZr3h89N
Author Name: MHAFEEZCARLY (@MHAFEEZCARLY)
Author Verified: False
Author Profile Image: https://pbs.twimg.com/profile_images/1960374925518106624/CzAesB0W_normal.jpg
Author URL: https://x.com/MHAFEEZCARLY
Tweet URL: https://x.com/MHAFEEZCARLY/status/1984935480836669717
Image URLs:
- https://pbs.twimg.com/media/G4voStrWQAABFO0.jpg
Likes: 2
Retweets: 0
Replies: 0


✅ Tweets successfully saved to 'tweets.csv'


**Reddit API**

In [7]:
!pip install praw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 3.4 MB/s eta 0:00:00


In [31]:
# Importing necessary libraries for Reddit API interaction
import praw
import datetime  # To format the timestamp from Reddit
import pandas as pd

# --- API Configuration ---
REDDIT_CLIENT_ID = 'utnSmiRRrv9ZDIlJ9LveMg'
REDDIT_CLIENT_SECRET = 'jzP9XRolmRsu3CNiDZmKF0IDDGcXpA'
REDDIT_USER_AGENT = 'python:moonsearcher:v1.0 (by /u/Dependent_Car_5879)'

# --- Search Parameters ---
try:
    question = " ".join(news_check).strip()
    num_results = 100  # Automatically take up to 100 results
except EOFError:
    print("\nNo input provided for search query. Exiting.")
    exit()

# --- Main Execution Block ---
if not REDDIT_CLIENT_ID or not REDDIT_CLIENT_SECRET or not question:
    print("ERROR: Client ID, Client Secret, and a search query are required.")
else:
    try:
        reddit = praw.Reddit(
            client_id=REDDIT_CLIENT_ID,
            client_secret=REDDIT_CLIENT_SECRET,
            user_agent=REDDIT_USER_AGENT,
        )

        search_results = reddit.subreddit("all").search(
            query=question,
            sort='new',
            limit=num_results
        )

        submissions = list(search_results)

        if not submissions:
            print("\nNo recent posts found for that query.")
        else:
            print(f"\n--- Found {len(submissions)} Posts ---\n")

            # --- List to collect data for DataFrame ---
            reddit_rows = []

            for submission in submissions:
                author_name = submission.author.name if submission.author else "[deleted]"
                post_url = f"https://www.reddit.com{submission.permalink}"
                author_url = f"https://www.reddit.com/user/{author_name}" if author_name != "[deleted]" else "N/A"
                published_time = datetime.datetime.fromtimestamp(submission.created_utc)

                # --- Print basic info ---
                print(f"Published Time: {published_time.strftime('%Y-%m-%d %H:%M:%S')}")
                print(f"Post Title: {submission.title}")
                print(f"Subreddit: r/{submission.subreddit.display_name}")
                print(f"Score: {submission.score} (Upvote Ratio: {submission.upvote_ratio * 100:.0f}%)")
                print(f"Author: u/{author_name}")
                print(f"Author URL: {author_url}")
                print(f"Post URL: {post_url}")

                post_text = submission.selftext.replace(chr(10), ' ') if submission.selftext else None

                # --- Extract image if any ---
                image_url = None
                if submission.url.endswith(('.jpg', '.jpeg', '.png', '.gif')):
                    image_url = submission.url
                elif hasattr(submission, 'preview'):
                    try:
                        image_url = submission.preview['images'][0]['source']['url']
                    except (KeyError, IndexError):
                        pass

                if image_url:
                    print(f"Image URL: {image_url}")
                else:
                    print("Image: None")

                print("\n" + "-" * 60 + "\n")

                # --- Add to DataFrame list ---
                reddit_rows.append({
                    "Published Time": published_time.strftime('%Y-%m-%d %H:%M:%S'),
                    "Post Title": submission.title,
                    "Subreddit": submission.subreddit.display_name,
                    "Score": submission.score,
                    "Upvote Ratio (%)": f"{submission.upvote_ratio * 100:.0f}",
                    "Author": author_name,
                    "Author URL": author_url,
                    "Post URL": post_url,
                    "Post Text": post_text,
                    "Image URL": image_url
                })

            # --- Convert posts to DataFrame and save to CSV ---
            df_reddit = pd.DataFrame(reddit_rows)
            df_reddit.to_csv('reddit.csv', index=False, encoding='utf-8')

            print("✅ Reddit posts successfully saved to 'reddit.csv'")

    except praw.exceptions.PRAWException as e:
        print("\n--- API ERROR ---")
        print("An error occurred with the Reddit API. This could be due to invalid credentials or a connection problem.")
        print(f"Error details: {e}")
    except Exception as e:
        print(f"\nAn unexpected error occurred.")
        print(f"Error details: {e}")


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.




--- Found 100 Posts ---

Published Time: 2025-11-06 13:57:22
Post Title: Trump wants South Africa out of the G20 as it gears up for world summit | US News
Subreddit: r/USANewsFlash
Score: 1 (Upvote Ratio: 100%)
Author: u/InnerLog5062
Author URL: https://www.reddit.com/user/InnerLog5062
Post URL: https://www.reddit.com/r/USANewsFlash/comments/1opzlqv/trump_wants_south_africa_out_of_the_g20_as_it/
Image URL: https://external-preview.redd.it/CWAjvH0Rcjzan1vkvaBFM9LtwOGhMNzHb-RRp2sSWkA.jpeg?auto=webp&s=2d486a193cccb87c5713d9a1ab85994c1188dace

------------------------------------------------------------

Published Time: 2025-11-06 13:53:00
Post Title: Anyone up for a chill coffee chat in South Delhi?
Subreddit: r/southdelhi
Score: 1 (Upvote Ratio: 100%)
Author: u/Aromatic_Drummer_756
Author URL: https://www.reddit.com/user/Aromatic_Drummer_756
Post URL: https://www.reddit.com/r/southdelhi/comments/1opzhyq/anyone_up_for_a_chill_coffee_chat_in_south_delhi/


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Image: None

------------------------------------------------------------

Published Time: 2025-11-06 13:52:48
Post Title: International conference to be held in New Delhi
Subreddit: r/CharteredAccountants
Score: 1 (Upvote Ratio: 100%)
Author: u/Moneyraj09
Author URL: https://www.reddit.com/user/Moneyraj09
Post URL: https://www.reddit.com/r/CharteredAccountants/comments/1opzhsc/international_conference_to_be_held_in_new_delhi/


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



...

Image: None

------------------------------------------------------------

Published Time: 2025-11-06 05:21:45
Post Title: Idk how to deal with mounted archers
Subreddit: r/aoe4
Score: 2 (Upvote Ratio: 67%)
Author: u/thanderrine
Author URL: https://www.reddit.com/user/thanderrine
Post URL: https://www.reddit.com/r/aoe4/comments/1opqgpg/idk_how_to_deal_with_mounted_archers/


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Image: None

------------------------------------------------------------

Published Time: 2025-11-06 05:19:05
Post Title: Someone said after looking at my kundali I will earn great but end up spending most of my wealth is it so?
Subreddit: r/vedicastrologyexperts
Score: 1 (Upvote Ratio: 67%)
Author: u/Sad_Positive8847
Author URL: https://www.reddit.com/user/Sad_Positive8847
Post URL: https://www.reddit.com/r/vedicastrologyexperts/comments/1opqeyc/someone_said_after_looking_at_my_kundali_i_will/
Image URL: https://i.redd.it/rio0sqmwmkzf1.png

------------------------------------------------------------

Published Time: 2025-11-06 05:07:10
Post Title: Alex Ovechkin scores his 900th NHL goal with the Washington Capitals | IHM News
Subreddit: r/IceHockeyMan
Score: 1 (Upvote Ratio: 100%)
Author: u/scelam1981
Author URL: https://www.reddit.com/user/scelam1981
Post URL: https://www.reddit.com/r/IceHockeyMan/comments/1opq6qg/alex_ovechkin_scores_his_900th_nhl_goal_with_the/


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Image: None

------------------------------------------------------------

Published Time: 2025-11-06 05:02:42
Post Title: Knights Among Us A Star Wars Tale
Subreddit: r/u_dakotaatsunrise
Score: 1 (Upvote Ratio: 100%)
Author: u/dakotaatsunrise
Author URL: https://www.reddit.com/user/dakotaatsunrise
Post URL: https://www.reddit.com/r/u_dakotaatsunrise/comments/1opq3p6/knights_among_us_a_star_wars_tale/
Image: None

------------------------------------------------------------

Published Time: 2025-11-06 05:02:09
Post Title: Knights Among Us
Subreddit: r/u_dakotaatsunrise
Score: 1 (Upvote Ratio: 100%)
Author: u/dakotaatsunrise
Author URL: https://www.reddit.com/user/dakotaatsunrise
Post URL: https://www.reddit.com/r/u_dakotaatsunrise/comments/1opq3c0/knights_among_us/
Image URL: https://i.redd.it/nh52zy2wjkzf1.jpeg

------------------------------------------------------------

✅ Reddit posts successfully saved to 'reddit.csv'


# **Data Preprocessing**

Real News

In [12]:
df

,Published Time,Title,Description,Image URL,Content,Source Name,Source URL,Source Country,Article URL
0,2025-11-06T01:07:33Z,Trump's G20 No-Show: Disdain for South African...,Former US President Donald Trump announced he ...,https://www.devdiscourse.com/remote.axd?https:...,Former US President Donald Trump has announced...,Devdiscourse,https://www.devdiscourse.com,in,https://www.devdiscourse.com/article/politics/...
1,2025-11-05T21:43:00Z,‘Shouldn’t Even Be In The Gs’: Trump To Skip S...,Donald Trump announces he will skip the 2025 G...,https://images.news18.com/ibnlive/uploads/2025...,US President Donald Trump has accused South Af...,News18,https://www.news18.com,in,https://www.news18.com/world/shouldnt-even-be-...
2,2025-11-05T19:15:54Z,Trump Challenges South Africa's G20 Status,Former U.S. President Donald Trump expressed t...,https://www.devdiscourse.com/remote.axd?https:...,"In a bold statement, former U.S. President Don...",Devdiscourse,https://www.devdiscourse.com,in,https://www.devdiscourse.com/article/politics/...
3,2025-11-05T15:26:14Z,Dr Dion George Urges G20 to Act on Declaration...,Speaking at the United for Wildlife Global Sum...,https://www.devdiscourse.com/remote.axd?https:...,"Minister of Forestry, Fisheries and the Enviro...",Devdiscourse,https://www.devdiscourse.com,in,https://www.devdiscourse.com/article/science-e...
4,2025-11-05T12:00:06Z,Gauteng Gears Up for G20: Panyaza Lesufi Confi...,“A comprehensive and resilient energy plan is ...,https://www.devdiscourse.com/remote.axd?https:...,With the world’s gaze soon to fall on Johannes...,Devdiscourse,https://www.devdiscourse.com,in,https://www.devdiscourse.com/article/law-order...


Twitter News

In [13]:
df_tweets

,Published Time,Tweet Text,Author Name,Author Username,Author Verified,Author Profile Image,Author URL,Tweet URL,Image URLs,Likes,Retweets,Replies
0,2025-11-06 13:04:30+00:00,"Trump Says South Africa Should Not Be In G20, ...",Femi Ayejusunle,FAyejusunle,False,https://pbs.twimg.com/profile_images/198522573...,https://x.com/FAyejusunle,https://x.com/FAyejusunle/status/1986419419237...,https://pbs.twimg.com/media/G5Et7WCXQAA0dJ2.jpg,0,0,0
1,2025-11-06 13:00:00+00:00,Trump Calls for South Africa’s Removal from G2...,Washington Eye,washington_EY,True,https://pbs.twimg.com/profile_images/196854331...,https://x.com/washington_EY,https://x.com/washington_EY/status/19864182877...,https://pbs.twimg.com/media/G5EhieUbIBEFAIl.jpg,0,0,0
2,2025-11-06 12:56:51+00:00,USA President Donald Trump has said South Afri...,African Hub,AfricanHub_,False,https://pbs.twimg.com/profile_images/153888503...,https://x.com/AfricanHub_,https://x.com/AfricanHub_/status/1986417494844...,https://pbs.twimg.com/media/G5EsLLKW4AASKU5.jpg,10,1,2
3,2025-11-06 12:47:36+00:00,The India-Middle East-Europe Economic Corridor...,Ramniwas Agrawal 🕉️ 🇮🇳,RamniwasAgrawal,False,https://pbs.twimg.com/profile_images/181104918...,https://x.com/RamniwasAgrawal,https://x.com/RamniwasAgrawal/status/198641516...,https://pbs.twimg.com/media/G5EqEb5bIAIxUFJ.jpg,0,0,0
4,2025-11-06 12:37:04+00:00,US President Donald Trump has announced he wil...,Judaeda Blanco,Judaeda3,False,https://pbs.twimg.com/profile_images/186578725...,https://x.com/Judaeda3,https://x.com/Judaeda3/status/1986412519166968165,https://pbs.twimg.com/media/G5Enp3aXsAAZqI7.jpg,0,1,0
5,2025-11-06 12:30:53+00:00,US President Donald Trump has declared that So...,AFRICA IS HOME GLOBAL,AfricaisHOME2,False,https://pbs.twimg.com/profile_images/127262214...,https://x.com/AfricaisHOME2,https://x.com/AfricaisHOME2/status/19864109593...,https://pbs.twimg.com/media/G5EmPNPWEAA4zjZ.jpg,0,0,0
6,2025-11-06 12:30:36+00:00,Gauteng is gearing up for the 2025 G20 Summit ...,Insight Mauritius News,MaurInsight,False,https://pbs.twimg.com/profile_images/194232418...,https://x.com/MaurInsight,https://x.com/MaurInsight/status/1986410889613...,https://pbs.twimg.com/media/G5El_o_awAASG5K.jpg,0,0,0
7,2025-11-06 12:13:51+00:00,RSVP Now for AEC’s #G20 Networking Reception |...,African Energy Chamber,energy_african,False,https://pbs.twimg.com/profile_images/126934291...,https://x.com/energy_african,https://x.com/energy_african/status/1986406676...,https://pbs.twimg.com/media/G5EiU5DWgAAD-mP.jpg,0,0,0
8,2025-11-06 12:11:41+00:00,U.S. President Donald Trump has announced he w...,African Hub,AfricanHub_,False,https://pbs.twimg.com/profile_images/153888503...,https://x.com/AfricanHub_,https://x.com/AfricanHub_/status/1986406128289...,https://pbs.twimg.com/media/G5EhzYCWMAAKDoO.jpg,28,5,3
9,2025-11-06 12:11:33+00:00,🤡 #Trump said that South Africa should be remo...,Arround The World,priyabharata203,False,https://pbs.twimg.com/profile_images/196596534...,https://x.com/priyabharata203,https://x.com/priyabharata203/status/198640609...,https://pbs.twimg.com/media/G5Ehrx5bIAMz8G8.jpg,0,0,0


Reddit News

In [14]:
df_reddit

,Published Time,Post Title,Subreddit,Score,Upvote Ratio (%),Author,Author URL,Post URL,Post Text,Image URL
0,2025-11-06 12:39:10,[Politics] - Ramaphosa's advisor denies resign...,AutoNewspaper,1,100,AutoNewspaperAdmin,https://www.reddit.com/user/AutoNewspaperAdmin,https://www.reddit.com/r/AutoNewspaper/comment...,None,None
1,2025-11-06 12:36:20,[Politics] - Ramaphosa's advisor denies resign...,IOLauto,1,100,AutoNewsAdmin,https://www.reddit.com/user/AutoNewsAdmin,https://www.reddit.com/r/IOLauto/comments/1opx...,None,None
2,2025-11-06 04:04:23,[Politics] - WATCH: Donald Trump insists South...,AutoNewspaper,1,100,AutoNewspaperAdmin,https://www.reddit.com/user/AutoNewspaperAdmin,https://www.reddit.com/r/AutoNewspaper/comment...,None,None
3,2025-11-06 04:04:22,[World] - WATCH: Donald Trump insists South Af...,AutoNewspaper,2,100,AutoNewspaperAdmin,https://www.reddit.com/user/AutoNewspaperAdmin,https://www.reddit.com/r/AutoNewspaper/comment...,None,None
4,2025-11-06 04:04:22,[ZA] - WATCH: Donald Trump insists South Afric...,AutoNewspaper,2,100,AutoNewspaperAdmin,https://www.reddit.com/user/AutoNewspaperAdmin,https://www.reddit.com/r/AutoNewspaper/comment...,None,None
...,...,...,...,...,...,...,...,...,...,...
95,2025-09-06 23:48:33,Trump Albanese meeting: US president to skip G...,u_IrreverentSunny,1,100,IrreverentSunny,https://www.reddit.com/user/IrreverentSunny,https://www.reddit.com/r/u_IrreverentSunny/com...,**Washington:** US President Donald Trump conf...,https://external-preview.redd.it/98LLQz1tomdz7...
96,2025-09-06 22:38:49,[PIL] #1786 9/6/2025,Purtle,1,100,Purtle,https://www.reddit.com/user/Purtle,https://www.reddit.com/r/Purtle/comments/1nacy...,"**Purtle's Internet Lineup for September 6th, ...",https://external-preview.redd.it/5J-y22aNQ5mW1...
97,2025-09-06 20:48:58,2026 G20 summit to be held at Trump National D...,YAPms,9,100,chia923,https://www.reddit.com/user/chia923,https://www.reddit.com/r/YAPms/comments/1naaen...,None,None
98,2025-09-06 16:10:57,Trump says US will host next year’s G20 summit...,news2,15,86,alternative_way_108,https://www.reddit.com/user/alternative_way_108,https://www.reddit.com/r/news2/comments/1na3h0...,None,https://external-preview.redd.it/eOFQggfjeR01H...


# Model Development

In [53]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [54]:
# --- Load data ---
try:
    df_gnews = pd.read_csv("gnews.csv")
    df_tweets = pd.read_csv("tweets.csv")
    df_reddit = pd.read_csv("reddit.csv")
except FileNotFoundError:
    print("❌ One or more CSV files not found. Please run the data collection scripts first.")
    exit()

In [71]:
# --- Prepare combined dataset ---
combined_df = pd.DataFrame()

In [70]:
if not df_gnews.empty:
    df_gnews['SourceType'] = 'trusted'
    df_gnews['Text'] = df_gnews[['Title', 'Description', 'Content']].fillna('').agg(' '.join, axis=1)
    combined_df = pd.concat([combined_df, df_gnews[['Text', 'SourceType']]], ignore_index=True)

if not df_tweets.empty:
    df_tweets['SourceType'] = 'social'
    df_tweets['Text'] = df_tweets['Tweet Text'].fillna('')
    combined_df = pd.concat([combined_df, df_tweets[['Text', 'SourceType']]], ignore_index=True)

if not df_reddit.empty:
    df_reddit['SourceType'] = 'social'
    df_reddit['Text'] = df_reddit[['Post Title', 'Post Text']].fillna('').agg(' '.join, axis=1)
    combined_df = pd.concat([combined_df, df_reddit[['Text', 'SourceType']]], ignore_index=True)

combined_df

,Text,SourceType
0,India poised to play key role at G20 Summit in...,trusted
1,India poised to play key role at G20 Summit in...,trusted
2,Is a Modi-Carney bilateral on the cards next m...,trusted
3,Seeing most Nigerians talking about this image...,social
4,Trump wants South Africa out of the G20 as it ...,social
...,...,...
203,Idk how to deal with mounted archers In aoe4 ...,social
204,Someone said after looking at my kundali I wil...,social
205,Alex Ovechkin scores his 900th NHL goal with t...,social
206,Knights Among Us A Star Wars Tale Knights Amon...,social


In [75]:
# --- Convert keyword list (from spaCy) to string ---
news_input = " ".join(news_check)
news_input

'India role G20 Summit'

In [76]:
# --- Vectorization ---
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(combined_df['Text'].tolist() + [news_input])

KeyError: 'Text'

In [59]:
# --- Calculate cosine and linear kernel similarity ---
cosine_sim_scores = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])[0]
linear_sim_scores = linear_kernel(tfidf_matrix[-1], tfidf_matrix[:-1])[0]

In [60]:
# Add both scores to combined_df
combined_df['Cosine Similarity'] = cosine_sim_scores
combined_df['Linear Similarity'] = linear_sim_scores

In [61]:
# --- Aggregate trusted vs social similarity ---
trusted_sim = combined_df.loc[combined_df['SourceType'] == 'trusted', ['Cosine Similarity', 'Linear Similarity']].mean().mean()
social_sim = combined_df.loc[combined_df['SourceType'] == 'social', ['Cosine Similarity', 'Linear Similarity']].mean().mean()

In [62]:
# --- Train lightweight classifiers for reliability check ---
X = tfidf_matrix[:-1]
y = (combined_df['SourceType'] == 'trusted').astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [63]:
# Logistic Regression
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)
log_prob = log_reg.predict_proba(tfidf_matrix[-1])[:, 1][0] * 100

In [64]:
# KNN classifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
knn_pred = knn.predict(tfidf_matrix[-1])[0]

In [65]:
# --- Final Weighted Decision ---
weights = {
    "cosine": 0.35,
    "linear": 0.25,
    "logistic": 0.25,
    "knn": 0.15
}

trust_score = (
    weights["cosine"] * trusted_sim * 100 +
    weights["linear"] * trusted_sim * 100 +
    weights["logistic"] * log_prob +
    weights["knn"] * (100 if knn_pred == 1 else 0)
)

In [66]:
# --- Classification threshold ---
classification = "✅ REAL NEWS" if trust_score >= 60 else "❌ POSSIBLY FAKE NEWS"

# --- Display results ---
print("\n📰 NEWS VERIFICATION RESULT 📰")
print(f"Entered News Keywords: {news_check}")
print(f"Cosine Similarity (Trusted): {trusted_sim*100:.2f}%")
print(f"Cosine Similarity (Social): {social_sim*100:.2f}%")
print(f"Logistic Regression Confidence: {log_prob:.2f}%")
print(f"Overall Trust Score: {trust_score:.2f}%")
print(f"🧾 Classification: {classification}")


📰 NEWS VERIFICATION RESULT 📰
Entered News Keywords: ['India', 'role', 'G20', 'Summit']
Cosine Similarity (Trusted): 35.02%
Cosine Similarity (Social): 2.86%
Logistic Regression Confidence: 3.78%
Overall Trust Score: 21.96%
🧾 Classification: ❌ POSSIBLY FAKE NEWS


In [50]:
# --- Related Social Sources (Potential Spreaders) ---
print("\n📢 Twitter Users Possibly Spreading It:")
if not df_tweets.empty:
    print(df_tweets[['Author Name', 'Author Username', 'Likes', 'Retweets', 'Replies', 'Tweet URL']].head(5).to_string(index=False))
else:
    print("No Twitter data available.")

print("\n🧠 Reddit Users Possibly Spreading It:")
if not df_reddit.empty:
    print(df_reddit[['Author', 'Subreddit', 'Score', 'Upvote Ratio (%)', 'Post URL']].head(5).to_string(index=False))
else:
    print("No Reddit data available.")


📢 Twitter Users Possibly Spreading It:
 Author Name Author Username  Likes  Retweets  Replies                                             Tweet URL
MHAFEEZCARLY    MHAFEEZCARLY      2         0        0 https://x.com/MHAFEEZCARLY/status/1984935480836669717

🧠 Reddit Users Possibly Spreading It:
              Author             Subreddit  Score  Upvote Ratio (%)                                                                                                         Post URL
        InnerLog5062          USANewsFlash      1               100            https://www.reddit.com/r/USANewsFlash/comments/1opzlqv/trump_wants_south_africa_out_of_the_g20_as_it/
Aromatic_Drummer_756            southdelhi      1               100           https://www.reddit.com/r/southdelhi/comments/1opzhyq/anyone_up_for_a_chill_coffee_chat_in_south_delhi/
          Moneyraj09  CharteredAccountants      1               100 https://www.reddit.com/r/CharteredAccountants/comments/1opzhsc/international_conference_to_b

In [78]:
# --- Machine Learning Based News Verification System ---
# Uses TF-IDF + Cosine Similarity + Logistic Regression + Decision Tree + SVM

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# --- Load CSV data ---
try:
    df_gnews = pd.read_csv("gnews.csv")
    df_tweets = pd.read_csv("tweets.csv")
    df_reddit = pd.read_csv("reddit.csv")
except FileNotFoundError:
    print("❌ Missing CSV files. Please ensure gnews.csv, tweets.csv, reddit.csv exist.")
    exit()

# --- Combine Data ---
combined_df = pd.DataFrame()

if not df_gnews.empty:
    df_gnews['Source'] = 'GNews'
    combined_df = pd.concat([combined_df, df_gnews[['Title', 'Description', 'Content', 'Source']].fillna('')])

if not df_tweets.empty:
    df_tweets['Source'] = 'Twitter'
    df_tweets = df_tweets.rename(columns={'Tweet Text': 'Content'})
    df_tweets['Title'], df_tweets['Description'] = '', ''
    combined_df = pd.concat([combined_df, df_tweets[['Title', 'Description', 'Content', 'Source']].fillna('')])

if not df_reddit.empty:
    df_reddit['Source'] = 'Reddit'
    df_reddit = df_reddit.rename(columns={'Post Title': 'Title', 'Post Text': 'Content'})
    df_reddit['Description'] = ''
    combined_df = pd.concat([combined_df, df_reddit[['Title', 'Description', 'Content', 'Source']].fillna('')])

# --- Merge text columns ---
combined_df['Text'] = combined_df.apply(
    lambda x: ' '.join([str(x.get('Title', '')), str(x.get('Description', '')), str(x.get('Content', ''))]),
    axis=1
)

# ✅ Label data for ML models:
# Assume: GNews = Real (1), Twitter/Reddit = Possibly Fake (0)
combined_df['Label'] = combined_df['Source'].apply(lambda x: 1 if x == 'GNews' else 0)

# --- Get user input ---
news_input = input("Enter the news to verify if it is real or fake:\n")

# --- TF-IDF Vectorization ---
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
tfidf_matrix = vectorizer.fit_transform(combined_df['Text'].tolist() + [news_input])

# --- Cosine Similarity ---
similarity_scores = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])[0]
combined_df['Similarity'] = similarity_scores
top_match = combined_df.sort_values(by='Similarity', ascending=False).head(1)
similarity_percent = float(top_match['Similarity'].values[0]) * 100

# --- Classification Rule ---
if similarity_percent >= 70:
    cosine_result = "✅ REAL NEWS"
else:
    cosine_result = "❌ FAKE NEWS"

print("\n🔍 COSINE SIMILARITY RESULT")
print(f"Top Match Source: {top_match['Source'].values[0]}")
print(f"Similarity: {similarity_percent:.2f}%")
print(f"Classification: {cosine_result}")

# --- Train ML Models ---
X_train, X_test, y_train, y_test = train_test_split(
    tfidf_matrix[:-1], combined_df['Label'], test_size=0.2, random_state=42
)

models = {
    "Logistic Regression": LogisticRegression(max_iter=200),
    "Decision Tree": DecisionTreeClassifier(),
    "SVM": SVC(probability=True)
}

print("\n🤖 MACHINE LEARNING MODEL RESULTS:")
for name, model in models.items():
    model.fit(X_train, y_train)
    pred = model.predict(tfidf_matrix[-1])
    result = "✅ REAL NEWS" if pred[0] == 1 else "❌ FAKE NEWS"
    print(f"{name}: {result}")

# --- Display related users spreading similar content ---
print("\n📢 Related Twitter Users:")
if not df_tweets.empty:
    print(df_tweets[['Author Name', 'Author Username', 'Author Verified', 'Likes', 'Retweets', 'Replies', 'Tweet URL']].head(5).to_string(index=False))
else:
    print("No Twitter data found.")

print("\n🧠 Related Reddit Users:")
if not df_reddit.empty:
    print(df_reddit[['Author', 'Subreddit', 'Score', 'Upvote Ratio (%)', 'Post URL']].head(5).to_string(index=False))
else:
    print("No Reddit data found.")


Enter the news to verify if it is real or fake:
ttps://preview.redd.it/w10ct04tzmzf1.png?width=600&format=png&auto=webp&s=80d210edb3f2fd3959385161ec129162a94259ca With pleasant weather and a jam-packed schedule, November in Delhi NCR feels like one long festival. From [**NykaaLand’s beauty carnival**](https://lalbeharitandon.com/collections/chikankari-new-arrivals-kurti) to Zamna’s EDM madness, the month promises unforgettable weekends for every mood. # 5. Final Word [**Delhi NCR’s event**](https://fillmefeed.com/top-10-events-to-attend-in-delhi-ncr-this-november-2025/) scene is redefining how India celebrates music, art, and comedy. If you haven’t updated your weekend calendar yet, you’re missing out on the city’s most dynamic month. Discover what’s happening, where to go, and who’s performing next — because the capital is calling, and the stage is already set.

🔍 COSINE SIMILARITY RESULT
Top Match Source: Reddit
Similarity: 74.79%
Classification: ✅ REAL NEWS

🤖 MACHINE LEARNING MODEL

# Experimentation & Optimization

In [84]:
# --- Machine Learning Based News Verification System ---
# TF-IDF + Cosine Similarity + Logistic Regression + Decision Tree + SVM
# Logic: Compare each Tweet/Reddit post with GNews content.

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# --- Load CSV data ---
try:
    df_gnews = pd.read_csv("gnews.csv")
    df_tweets = pd.read_csv("tweets.csv")
    df_reddit = pd.read_csv("reddit.csv")
except FileNotFoundError:
    print("❌ Missing CSV files. Please ensure gnews.csv, tweets.csv, reddit.csv exist.")
    exit()

# --- Prepare GNews Data ---
df_gnews['Source'] = 'GNews'
df_gnews['Combined'] = df_gnews[['Title', 'Description', 'Content']].fillna('').agg(' '.join, axis=1)

# --- Prepare Twitter Data ---
if not df_tweets.empty:
    df_tweets['Source'] = 'Twitter'
    df_tweets['Combined'] = df_tweets['Tweet Text'].fillna('')
else:
    df_tweets = pd.DataFrame(columns=['Source', 'Combined'])

# --- Prepare Reddit Data ---
if not df_reddit.empty:
    df_reddit['Source'] = 'Reddit'
    df_reddit['Combined'] = df_reddit[['Post Title', 'Post Text']].fillna('').agg(' '.join, axis=1)
else:
    df_reddit = pd.DataFrame(columns=['Source', 'Combined'])

# --- TF-IDF Vectorization on all content ---
all_texts = df_gnews['Combined'].tolist() + df_tweets['Combined'].tolist() + df_reddit['Combined'].tolist()
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
tfidf_matrix = vectorizer.fit_transform(all_texts)

# Split back indexes
gnews_vectors = tfidf_matrix[:len(df_gnews)]
tweet_vectors = tfidf_matrix[len(df_gnews):len(df_gnews) + len(df_tweets)]
reddit_vectors = tfidf_matrix[len(df_gnews) + len(df_tweets):]

# --- Compare Twitter posts to GNews ---
tweet_labels = []
tweet_similarities = []

if len(df_tweets) > 0:
    sim_tweet_gnews = cosine_similarity(tweet_vectors, gnews_vectors)
    for sim_row in sim_tweet_gnews:
        max_sim = sim_row.max()
        tweet_similarities.append(max_sim)
        label = 1 if max_sim >= 0.6 else 0  # Real if ≥ 60% similarity
        tweet_labels.append(label)

    df_tweets['Max Similarity'] = tweet_similarities
    df_tweets['Label'] = tweet_labels

# --- Compare Reddit posts to GNews ---
reddit_labels = []
reddit_similarities = []

if len(df_reddit) > 0:
    sim_reddit_gnews = cosine_similarity(reddit_vectors, gnews_vectors)
    for sim_row in sim_reddit_gnews:
        max_sim = sim_row.max()
        reddit_similarities.append(max_sim)
        label = 1 if max_sim >= 0.7 else 0
        reddit_labels.append(label)

    df_reddit['Max Similarity'] = reddit_similarities
    df_reddit['Label'] = reddit_labels

# --- Combine labeled data for ML ---
df_combined = pd.concat([df_gnews[['Combined']].assign(Label=1),
                         df_tweets[['Combined', 'Label']],
                         df_reddit[['Combined', 'Label']]],
                        ignore_index=True)

# --- Train-Test Split ---
X_train, X_test, y_train, y_test = train_test_split(
    df_combined['Combined'], df_combined['Label'], test_size=0.2, random_state=42
)

# --- Vectorize for ML Models ---
vectorizer_ml = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_vec = vectorizer_ml.fit_transform(X_train)
X_test_vec = vectorizer_ml.transform(X_test)

# --- Train Models ---
models = {
    "Logistic Regression": LogisticRegression(max_iter=200),
    "Decision Tree": DecisionTreeClassifier(),
    "SVM": SVC(probability=True)
}

print("\n🤖 MACHINE LEARNING MODEL RESULTS (on known dataset):")
for name, model in models.items():
    model.fit(X_train_vec, y_train)
    acc = model.score(X_test_vec, y_test) * 100
    print(f"{name}: Accuracy = {acc:.2f}%")

# --- User Input ---
news_input = news

# --- Compare user input to GNews ---
user_vec = vectorizer.transform([news_input])
similarity_to_gnews = cosine_similarity(user_vec, gnews_vectors).max()

classification = "✅ REAL NEWS" if similarity_to_gnews >= 0.7 else "❌ FAKE NEWS"
print(f"\n🔍 COSINE SIMILARITY RESULT:")
print(f"Similarity with trusted GNews: {similarity_to_gnews * 100:.2f}%")

# --- Model predictions for user input ---
user_vec_ml = vectorizer_ml.transform([news_input])

print("\n📊 ML Model Predictions:")
for name, model in models.items():
    pred = model.predict(user_vec_ml)[0]
    print(f"{name}: {'✅ REAL NEWS' if pred == 1 else '❌ FAKE NEWS'}")

# --- Display Related Twitter/Reddit Users spreading similar content ---
print("\n📢 Related Twitter Users spreading similar content:")
if not df_tweets.empty:
    similar_tweets = df_tweets[df_tweets['Label'] == 0].head(5)
    if not similar_tweets.empty:
        print(similar_tweets[['Author Name', 'Author Username', 'Author Verified',
                              'Likes', 'Retweets', 'Replies', 'Tweet URL', 'Max Similarity']].to_string(index=False))
    else:
        print("✅ No fake or mismatched tweets found.")
else:
    print("No Twitter data available.")

print("\n🧠 Related Reddit Users spreading similar content:")
if not df_reddit.empty:
    similar_reddit = df_reddit[df_reddit['Label'] == 0].head(5)
    if not similar_reddit.empty:
        print(similar_reddit[['Author', 'Subreddit', 'Score', 'Upvote Ratio (%)',
                              'Post URL', 'Max Similarity']].to_string(index=False))
    else:
        print("✅ No fake or mismatched Reddit posts found.")
else:
    print("No Reddit data available.")



🤖 MACHINE LEARNING MODEL RESULTS (on known dataset):
Logistic Regression: Accuracy = 95.24%
Decision Tree: Accuracy = 100.00%
SVM: Accuracy = 95.24%

🔍 COSINE SIMILARITY RESULT:
Similarity with trusted GNews: 91.69%

📊 ML Model Predictions:
Logistic Regression: ❌ FAKE NEWS
Decision Tree: ✅ REAL NEWS
SVM: ❌ FAKE NEWS

📢 Related Twitter Users spreading similar content:
 Author Name Author Username  Author Verified  Likes  Retweets  Replies                                             Tweet URL  Max Similarity
MHAFEEZCARLY    MHAFEEZCARLY            False      2         0        0 https://x.com/MHAFEEZCARLY/status/1984935480836669717        0.149411

🧠 Related Reddit Users spreading similar content:
              Author             Subreddit  Score  Upvote Ratio (%)                                                                                                         Post URL  Max Similarity
        InnerLog5062          USANewsFlash      1               100            https://www.reddit